In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance

def transform_image(image_path, flip_horizontal=False, shear_x=0, shear_y=0, saturation_change=0):
    """
    이미지를 변환하는 함수
    
    Parameters:
    image_path (str): 이미지 파일 경로
    flip_horizontal (bool): 수평 반전 여부
    shear_x (float): 수평 기울기 각도 (-10 ~ 10)
    shear_y (float): 수직 기울기 각도 (-10 ~ 10)
    saturation_change (float): 채도 변화량 (-25 ~ 25)
    
    Returns:
    PIL.Image: 변환된 이미지
    """
    # 이미지 로드
    img = cv2.imread(image_path)
    
    # BGR을 RGB로 변환
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # 1. 수평 반전
    if flip_horizontal:
        img = cv2.flip(img, 1)
    
    # 2. 기울이기 변환 (Shear transformation)
    rows, cols = img.shape[:2]
    
    # 기울기 각도를 라디안으로 변환
    shear_x_rad = np.deg2rad(shear_x)
    shear_y_rad = np.deg2rad(shear_y)
    
    # 변환 행렬 생성
    shear_matrix = np.float32([
        [1, np.tan(shear_x_rad), 0],
        [np.tan(shear_y_rad), 1, 0]
    ])
    
    # 이미지 중심점 계산
    center_x, center_y = cols/2, rows/2
    
    # 변환 행렬 조정
    shear_matrix[0, 2] = -center_x * np.tan(shear_x_rad)
    shear_matrix[1, 2] = -center_y * np.tan(shear_y_rad)
    
    # 기울이기 변환 적용
    img = cv2.warpAffine(img, shear_matrix, (cols, rows))
    
    # 3. 채도 조절
    # OpenCV 이미지를 PIL 이미지로 변환
    pil_img = Image.fromarray(img)
    
    # 채도 조절
    saturation_factor = 1 + (saturation_change / 100)
    enhancer = ImageEnhance.Color(pil_img)
    pil_img = enhancer.enhance(saturation_factor)
    
    return pil_img

def save_transformed_image(input_path, output_path, flip_horizontal=False, shear_x=0, shear_y=0, saturation_change=0):
    """
    변환된 이미지를 저장하는 함수
    
    Parameters:
    input_path (str): 입력 이미지 경로
    output_path (str): 출력 이미지 경로
    """
    transformed_img = transform_image(
        input_path,
        flip_horizontal,
        shear_x,
        shear_y,
        saturation_change
    )
    transformed_img.save(output_path)

# 사용 예시
if __name__ == "__main__":
    # 이미지 파일 경로 설정
    input_image_path = "C:/Users\\/AI-LHJ/Desktop/YOLOv11/test.jpg"  # 여기에 실제 이미지 경로를 입력하세요
    output_image_path = "transformed_image.jpg"
    
    # 이미지 변환 및 저장
    save_transformed_image(
        input_image_path,
        output_image_path,
        flip_horizontal=True,  # 수평 반전 적용
        shear_x=5,            # 수평 기울기 5도
        shear_y=-5,           # 수직 기울기 -5도
        saturation_change=15  # 채도 15% 증가
    )

In [4]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import os

def transform_image(image_path, flip_horizontal=False, shear_x=0, shear_y=0, saturation_change=0):
    """이미지 변환 함수"""
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"이미지 파일을 찾을 수 없습니다: {image_path}")
    
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"이미지를 불러올 수 없습니다: {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # 1. 수평 반전
    if flip_horizontal:
        img = cv2.flip(img, 1)
    
    # 2. 기울이기 변환
    rows, cols = img.shape[:2]
    
    shear_x_rad = np.deg2rad(shear_x)
    shear_y_rad = np.deg2rad(shear_y)
    
    shear_matrix = np.float32([
        [1, np.tan(shear_x_rad), 0],
        [np.tan(shear_y_rad), 1, 0]
    ])
    
    center_x, center_y = cols/2, rows/2
    shear_matrix[0, 2] = -center_x * np.tan(shear_x_rad)
    shear_matrix[1, 2] = -center_y * np.tan(shear_y_rad)
    
    img = cv2.warpAffine(img, shear_matrix, (cols, rows))
    
    # 3. 채도 조절
    pil_img = Image.fromarray(img)
    saturation_factor = 1 + (saturation_change / 100)
    enhancer = ImageEnhance.Color(pil_img)
    pil_img = enhancer.enhance(saturation_factor)
    
    return pil_img

def create_all_transformations(input_path, output_dir):
    """
    모든 변환을 적용하고 개별 이미지로 저장하는 함수
    """
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        # 원본 이미지 저장
        original_img = Image.open(input_path)
        original_img.save(os.path.join(output_dir, "0_original.jpg"))
        print("원본 이미지 저장 완료")

        # 1. 수평 반전
        flipped = transform_image(input_path, flip_horizontal=True)
        flipped.save(os.path.join(output_dir, "1_flipped.jpg"))
        print("수평 반전 이미지 저장 완료")

        # 2. 수평 기울기 (Shear X)
        shear_x_minus = transform_image(input_path, shear_x=-10)
        shear_x_minus.save(os.path.join(output_dir, "2_shear_x_minus10.jpg"))
        
        shear_x_plus = transform_image(input_path, shear_x=10)
        shear_x_plus.save(os.path.join(output_dir, "3_shear_x_plus10.jpg"))
        print("수평 기울기 이미지 저장 완료")

        # 3. 수직 기울기 (Shear Y)
        shear_y_minus = transform_image(input_path, shear_y=-10)
        shear_y_minus.save(os.path.join(output_dir, "4_shear_y_minus10.jpg"))
        
        shear_y_plus = transform_image(input_path, shear_y=10)
        shear_y_plus.save(os.path.join(output_dir, "5_shear_y_plus10.jpg"))
        print("수직 기울기 이미지 저장 완료")

        # 4. 채도 변경
        saturation_minus = transform_image(input_path, saturation_change=-25)
        saturation_minus.save(os.path.join(output_dir, "6_saturation_minus25.jpg"))
        
        saturation_plus = transform_image(input_path, saturation_change=25)
        saturation_plus.save(os.path.join(output_dir, "7_saturation_plus25.jpg"))
        print("채도 변경 이미지 저장 완료")

        print("\n모든 변환 이미지가 성공적으로 저장되었습니다.")
        print(f"저장된 디렉토리: {output_dir}")
        
        # 생성된 파일 목록 출력
        print("\n생성된 파일 목록:")
        for filename in sorted(os.listdir(output_dir)):
            print(f"- {filename}")

    except Exception as e:
        print(f"에러 발생: {str(e)}")

# 사용 예시
if __name__ == "__main__":
    # 이미지 경로 설정
    input_image_path = "C:/Users\\/AI-LHJ/Desktop/YOLOv11/test.jpg" # 입력 이미지 경로
    output_directory = "C:/Users\\/AI-LHJ/Desktop/YOLOv11/transformed_images"  # 출력 디렉토리
    
    # 모든 변환 실행
    create_all_transformations(input_image_path, output_directory)

원본 이미지 저장 완료
수평 반전 이미지 저장 완료
수평 기울기 이미지 저장 완료
수직 기울기 이미지 저장 완료
채도 변경 이미지 저장 완료

모든 변환 이미지가 성공적으로 저장되었습니다.
저장된 디렉토리: C:/Users\/AI-LHJ/Desktop/YOLOv11/transformed_images

생성된 파일 목록:
- 0_original.jpg
- 1_flipped.jpg
- 2_shear_x_minus10.jpg
- 3_shear_x_plus10.jpg
- 4_shear_y_minus10.jpg
- 5_shear_y_plus10.jpg
- 6_saturation_minus25.jpg
- 7_saturation_plus25.jpg
